# 📝 HOMEWORK 1 — Great Expectations (Amazon Sales Dataset)

Bu notebook, Amazon Sales dataseti üzerinde **Great Expectations Core v1.0+** kullanarak data quality validation yapar ve Slack'e bildirim gönderir.

## 🎯 Öğrenme Hedefleri
- GX Core v1.0+ code-first workflow kullanımı
- Ephemeral Data Context oluşturma (dosya yazmadan)
- Expectation Suite tanımlama
- Validation çalıştırma ve sonuçları yorumlama
- Slack entegrasyonu

## 📦 Dataset'teki Kasıtlı Hatalar

| Satır | Hata | Yakalanacak Expectation |
|-------|------|-------------------------|
| 8 | `Amount` boş (NULL) | ExpectColumnValuesToBeBetween |
| 9 | `ship-city` = "Chennai" (küçük harf) | ❌ Bonus challenge |
| 11 | `Order ID` boş (NULL) | ExpectColumnValuesToNotBeNull |
| 12 | `Qty` = -1 (negatif) | ExpectColumnValuesToBeBetween |
| 13 | `currency` = "USD", `ship-country` = "US" | ExpectColumnValuesToBeInSet |
| 14 | `Date` = "invalid-date" | ExpectColumnValuesToMatchRegex |

---
## 1️⃣ Kurulum

Gerekli kütüphaneleri yükleyelim.

In [ ]:
# Kurulum (sadece ilk seferde çalıştırın)
# !pip install great_expectations pandas requests

In [ ]:
# Import'lar
import great_expectations as gx
import pandas as pd
import requests
import json
from datetime import datetime

# GX versiyonunu kontrol et (v1.0+ olmalı)
print(f"Great Expectations Version: {gx.__version__}")

---
## 2️⃣ Konfigürasyon

Validation kuralları için beklenen değerleri tanımlayalım.

In [ ]:
# Dosya yolu
CSV_PATH = "amazon_sales.csv"

# Slack Webhook URL (kendi URL'inizi yazın)
SLACK_WEBHOOK_URL = "YOUR_SLACK_WEBHOOK_URL"

# Beklenen değerler
VALID_STATUSES = [
    "Cancelled",
    "Shipped",
    "Shipped - Delivered to Buyer",
    "Pending",
    "Shipped - Returned to Seller",
    "Shipped - Rejected by Buyer",
    "Shipped - Returning to Seller",
    "Shipped - Out for Delivery",
    "Shipped - Picked Up"
]

VALID_FULFILMENT = ["Merchant", "Amazon"]
VALID_CURRENCIES = ["INR"]
VALID_COUNTRIES = ["IN"]

---
## 3️⃣ Veriyi Yükle ve İncele

Önce datasetin yapısını anlayalım.

In [ ]:
# CSV'yi yükle
df = pd.read_csv(CSV_PATH)

print(f"📊 Dataset Boyutu: {df.shape[0]} satır, {df.shape[1]} sütun")
print(f"\n📋 Sütunlar: {list(df.columns)}")

In [ ]:
# İlk birkaç satıra bakalım
df.head(10)

In [ ]:
# Veri tipleri ve null değerler
df.info()

In [ ]:
# Temel istatistikler
df.describe()

### 🔍 Soru: Yukarıdaki çıktılarda herhangi bir sorun görüyor musunuz?

*Düşünün ve aşağıda validation ile kontrol edelim...*

---
## 4️⃣ Great Expectations Context Oluştur

GX Core v1.0+ ile **Ephemeral Data Context** kullanıyoruz. Bu, herhangi bir dosya yazmadan bellekte çalışır.

In [ ]:
# Ephemeral Data Context oluştur (dosya yazmaz!)
context = gx.get_context()

# Context tipini doğrula
print(f"✅ Context Type: {type(context).__name__}")

---
## 5️⃣ Data Source ve Data Asset Tanımla

GX'e verimizi tanıtalım.

In [ ]:
# Pandas Data Source ekle
data_source = context.data_sources.add_pandas(name="amazon_sales_source")
print(f"✅ Data Source oluşturuldu: {data_source.name}")

# DataFrame Asset ekle
data_asset = data_source.add_dataframe_asset(name="amazon_sales_asset")
print(f"✅ Data Asset oluşturuldu: {data_asset.name}")

# Batch Definition oluştur (tüm DataFrame'i tek batch olarak al)
batch_definition = data_asset.add_batch_definition_whole_dataframe(name="full_dataframe")
print(f"✅ Batch Definition oluşturuldu: {batch_definition.name}")

---
## 6️⃣ Expectation Suite Oluştur

Şimdi data quality kurallarımızı (Expectations) tanımlayalım.

In [ ]:
# Expectation Suite oluştur
suite = gx.ExpectationSuite(name="amazon_sales_suite")

# Suite'i context'e ekle
suite = context.suites.add(suite)
print(f"✅ Expectation Suite oluşturuldu: {suite.name}")

### 6.1 Order ID Kontrolleri

In [ ]:
# Order ID: NULL olmamalı
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToNotBeNull(column="Order ID")
)
print("✅ Expectation eklendi: Order ID - Not Null")

# Order ID: Unique olmalı
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeUnique(column="Order ID")
)
print("✅ Expectation eklendi: Order ID - Unique")

### 6.2 Qty (Miktar) Kontrolü

In [ ]:
# Qty: 0 veya daha büyük olmalı (negatif olamaz)
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeBetween(
        column="Qty",
        min_value=0,
        max_value=None  # Üst limit yok
    )
)
print("✅ Expectation eklendi: Qty >= 0")

### 6.3 Amount (Tutar) Kontrolü

In [ ]:
# Amount: 0 veya daha büyük olmalı
# Not: NULL değerler de başarısız sayılacak
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeBetween(
        column="Amount",
        min_value=0,
        max_value=None
    )
)
print("✅ Expectation eklendi: Amount >= 0")

### 6.4 Kategorik Değer Kontrolleri

In [ ]:
# Status: Belirlenen değerler içinde olmalı
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeInSet(
        column="Status",
        value_set=VALID_STATUSES
    )
)
print("✅ Expectation eklendi: Status - Valid Values")

# Fulfilment: Merchant veya Amazon olmalı
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeInSet(
        column="Fulfilment",
        value_set=VALID_FULFILMENT
    )
)
print("✅ Expectation eklendi: Fulfilment - Valid Values")

# Currency: INR olmalı
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeInSet(
        column="currency",
        value_set=VALID_CURRENCIES
    )
)
print("✅ Expectation eklendi: Currency = INR")

# Ship Country: IN olmalı
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToBeInSet(
        column="ship-country",
        value_set=VALID_COUNTRIES
    )
)
print("✅ Expectation eklendi: Ship Country = IN")

### 6.5 Tarih Format Kontrolü

In [ ]:
# Date: MM-DD-YY formatında olmalı
suite.add_expectation(
    gx.expectations.ExpectColumnValuesToMatchRegex(
        column="Date",
        regex=r"^\d{2}-\d{2}-\d{2}$"  # MM-DD-YY
    )
)
print("✅ Expectation eklendi: Date Format (MM-DD-YY)")

In [ ]:
# Toplam expectation sayısı
print(f"\n📋 Toplam Expectation Sayısı: {len(suite.expectations)}")

---
## 7️⃣ Validation Definition Oluştur ve Çalıştır

In [ ]:
# Validation Definition oluştur
validation_definition = gx.ValidationDefinition(
    name="amazon_sales_validation",
    data=batch_definition,
    suite=suite
)

# Context'e ekle
validation_definition = context.validation_definitions.add(validation_definition)
print(f"✅ Validation Definition oluşturuldu: {validation_definition.name}")

In [ ]:
# Validation'ı çalıştır
print("🔄 Validation çalıştırılıyor...\n")

results = validation_definition.run(
    batch_parameters={"dataframe": df}
)

print("✅ Validation tamamlandı!")

---
## 8️⃣ Sonuçları İncele

In [ ]:
# Genel sonuç
print("=" * 60)
print("   VALIDATION SONUCU")
print("=" * 60)

if results.success:
    print("\n   🎉 TÜM KONTROLLER BAŞARILI!")
else:
    print("\n   ❌ BAZI KONTROLLER BAŞARISIZ!")

print("=" * 60)

In [ ]:
# Detaylı sonuçları işle
results_dict = results.to_json_dict()
expectation_results = results_dict.get("results", [])

passed = []
failed = []

for exp_result in expectation_results:
    exp_config = exp_result.get("expectation_config", {})
    exp_type = exp_config.get("type", "Unknown")
    column = exp_config.get("kwargs", {}).get("column", "N/A")
    success = exp_result.get("success", False)
    result_detail = exp_result.get("result", {})
    
    info = {
        "expectation": exp_type,
        "column": column,
        "success": success,
        "result": result_detail
    }
    
    if success:
        passed.append(info)
    else:
        failed.append(info)

print(f"\n📊 Özet:")
print(f"   ✅ Başarılı: {len(passed)}")
print(f"   ❌ Başarısız: {len(failed)}")
print(f"   📋 Toplam: {len(expectation_results)}")

In [ ]:
# Başarısız kontrolleri detaylı göster
if failed:
    print("\n" + "=" * 60)
    print("   ❌ BAŞARISIZ KONTROLLER")
    print("=" * 60)
    
    for f in failed:
        print(f"\n📌 {f['expectation']}")
        print(f"   Sütun: {f['column']}")
        
        result = f.get('result', {})
        unexpected_count = result.get('unexpected_count', 0)
        unexpected_percent = result.get('unexpected_percent', 0)
        unexpected_values = result.get('partial_unexpected_list', [])
        
        if unexpected_count:
            print(f"   Hatalı Kayıt Sayısı: {unexpected_count}")
            print(f"   Hatalı Kayıt Oranı: {unexpected_percent:.2f}%")
        if unexpected_values:
            print(f"   Örnek Hatalı Değerler: {unexpected_values[:5]}")
else:
    print("\n🎉 Tüm kontroller başarılı!")

In [ ]:
# Başarılı kontrolleri göster
if passed:
    print("\n" + "=" * 60)
    print("   ✅ BAŞARILI KONTROLLER")
    print("=" * 60)
    
    for p in passed:
        print(f"   ✓ {p['expectation']} (Sütun: {p['column']})")

---
## 9️⃣ Slack Bildirimi Gönder

Validation sonuçlarını Slack'e gönderelim.

In [ ]:
def send_slack_notification(webhook_url, overall_success, passed_count, failed_count, failed_list):
    """
    Slack'e validation sonuçlarını gönder.
    """
    if webhook_url == "YOUR_SLACK_WEBHOOK_URL":
        print("⚠️  Slack bildirimi atlandı (webhook URL yapılandırılmamış)")
        print("   Kendi Slack Webhook URL'inizi SLACK_WEBHOOK_URL değişkenine yazın.")
        return False
    
    # Emoji ve renk
    if overall_success:
        emoji = "✅"
        color = "#36a64f"
        status_text = "PASSED"
    else:
        emoji = "❌"
        color = "#dc3545"
        status_text = "FAILED"
    
    # Failed detayları
    failed_details = ""
    if failed_list:
        failed_details = "\n".join([
            f"• *{f['expectation']}* (Sütun: `{f['column']}`)"
            for f in failed_list
        ])
    
    # Mesaj oluştur
    message = {
        "attachments": [
            {
                "color": color,
                "blocks": [
                    {
                        "type": "header",
                        "text": {
                            "type": "plain_text",
                            "text": f"{emoji} Data Quality Validation {status_text}",
                            "emoji": True
                        }
                    },
                    {
                        "type": "section",
                        "fields": [
                            {"type": "mrkdwn", "text": f"*Dataset:*\nAmazon Sales"},
                            {"type": "mrkdwn", "text": f"*Zaman:*\n{datetime.now().strftime('%Y-%m-%d %H:%M')}"},
                            {"type": "mrkdwn", "text": f"*Başarılı:*\n{passed_count} ✓"},
                            {"type": "mrkdwn", "text": f"*Başarısız:*\n{failed_count} ✗"}
                        ]
                    }
                ]
            }
        ]
    }
    
    # Failed varsa ekle
    if failed_details:
        message["attachments"][0]["blocks"].append({
            "type": "section",
            "text": {
                "type": "mrkdwn",
                "text": f"*Başarısız Kontroller:*\n{failed_details}"
            }
        })
    
    # Gönder
    try:
        response = requests.post(
            webhook_url,
            data=json.dumps(message),
            headers={"Content-Type": "application/json"}
        )
        
        if response.status_code == 200:
            print("✅ Slack bildirimi başarıyla gönderildi!")
            return True
        else:
            print(f"❌ Slack bildirimi başarısız: {response.status_code}")
            return False
            
    except Exception as e:
        print(f"❌ Slack bildirimi hatası: {e}")
        return False

In [ ]:
# Slack bildirimi gönder
send_slack_notification(
    webhook_url=SLACK_WEBHOOK_URL,
    overall_success=results.success,
    passed_count=len(passed),
    failed_count=len(failed),
    failed_list=failed
)

---
## 🔟 CI/CD için Exit Code

GitHub Actions veya Jenkins'te kullanmak için:

In [ ]:
# CI/CD için (bu hücreyi script olarak çalıştırırken kullanın)
# if not results.success:
#     print("⚠️ Validation başarısız! Exit code: 1")
#     # exit(1)  # Uncomment for CI/CD
# else:
#     print("🎉 Tüm validasyonlar başarılı!")
#     # exit(0)  # Uncomment for CI/CD

---
## 📚 Özet

Bu notebook'ta öğrendiklerimiz:

1. **GX Core v1.0+ API** kullanımı (code-first, dosyasız)
2. **Ephemeral Data Context** oluşturma
3. **Data Source → Data Asset → Batch Definition** zinciri
4. **Expectation Suite** tanımlama
5. **Validation Definition** ile validation çalıştırma
6. **Slack entegrasyonu** ile bildirim gönderme

### 🏋️ Bonus Alıştırmalar

1. `ship-city` sütunundaki büyük/küçük harf tutarsızlığını yakalayacak bir expectation ekleyin
2. `ship-postal-code` için 6 haneli format kontrolü ekleyin
3. `Status` = "Cancelled" iken `Qty` = 0 olmalı kuralını ekleyin (compound expectation)

---
## 🔗 Kaynaklar

- [GX Core Documentation](https://docs.greatexpectations.io/docs/core/introduction/)
- [Try GX Core](https://docs.greatexpectations.io/docs/core/introduction/try_gx/)
- [Connect to DataFrame Data](https://docs.greatexpectations.io/docs/core/connect_to_data/dataframes/)
- [Expectation Gallery](https://greatexpectations.io/expectations/)